In [195]:
import pandas as pd

1. Look at the PBA50+ BAUS zoning output for parcels

In [196]:
# new buildings output for PBA50+ DBP
# 2050 output because as built_dua updates the max, max dua calculation is updated
pz = pd.read_csv("M:/urban_modeling/baus/PBA50Plus/Draft Blueprint Datasets/\
PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_rerun_v3/core_summaries/PBA50Plus_Draft_Blueprint_v8_znupd_nodevfix_rerun_v3_parcel_summary_2050.csv")

2. Add growth geography variables for understanding parcels' location

In [197]:
# parcels geography input file
pg = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcels_geography_2024_02_14.csv")

pz = pz.merge(pg[['PARCEL_ID', 'juris', 'gg_id', 'pda_id', 'tra_id', 'hra_id']], left_on='parcel_id', right_on='PARCEL_ID', how='left')

3. add the max built DUA/FAR in a parcel's TAZ, used in the BAUS zoning processing

In [198]:
# parcel to TAZ crosswalk
ptaz = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/2020_08_17_parcel_to_taz1454sub.csv")
pz = pz.merge(ptaz[['PARCEL_ID', 'ZONE_ID']], left_on='parcel_id', right_on='PARCEL_ID', how='left')

max_dua_builtdua_taz = pz.built_dua.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtdua_taz'] = pz['ZONE_ID'].map(max_dua_builtdua_taz).fillna(4)

max_far_builtdua_taz = pz.built_far.groupby(pz.ZONE_ID).max().fillna(4)
pz['max_dua_builtfar_taz'] = pz['ZONE_ID'].map(max_far_builtdua_taz).fillna(4)

4. Add the base zoning to parcels in order to lookup the corresponding zoning input

In [199]:
# latest parcels zoning file
pzon = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_parcels_2024-04-30.csv")
# latest parcel lookup file
zl = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/zoning/zoning_lookup_2024-04-30.csv")
zl = zl.rename(columns={"name": "zoning_name", "max_dua":"max_dua_zoning", "max_far": "max_far_zoning"})

# merge the zoning lookup to parcels
pz = pz.merge(pzon[['PARCEL_ID', 'zoning_id']], left_on='parcel_id', right_on='PARCEL_ID', how='left')\
.merge(zl[['id', 'zoning_name', 'max_dua_zoning', 'max_far_zoning']], left_on='zoning_id', right_on='id', how='left')

5. Inspect the BAUS zonign based on the processing steps

BAUS processing steps / zoning variables in the table:
- **max_dua_zoning** is the input zoning
- **max_dua_imputation** is zoning with imputation and residential allowance applied
- **max_dua_imputation_upzoning** is zoning with imputation and upzoning and residential allowance applied
- **maz_dua_builtfar_taz** is the highest max DUA in the TAZ
- **max_dua** is zoning with imputation and upzoning, residenital allowance, nodev modification, outside of urban area modification, \
and lowest built density modification applied

### DUA

In [200]:
print("Number of parcels with DUA > 40 is {}".format(len(pz.loc[pz.max_dua > 40])))
dua = pz.loc[pz.max_dua > 40][['parcel_id', 'zoning_name', 'juris', 'tra_id', 'pda_id', 'max_dua_zoning', 
                               'max_dua_imputation', 'max_dua_imputation_upzoning', 'max_dua_builtdua_taz', 'max_dua']].\
                                sort_values(by='max_dua', ascending=False)

Number of parcels with DUA > 40 is 243039


#### DUA from upzoning

In [201]:
dua_upzoning = dua.loc[(dua.max_dua_imputation_upzoning > dua.max_dua_imputation) & (dua.max_dua_imputation_upzoning == dua.max_dua)]
print("Number of parcels with upzoning > 200 is {}".format(len(dua_upzoning.loc[dua_upzoning.max_dua > 200])))
dua_upzoning.loc[dua_upzoning.max_dua > 200].head()

Number of parcels with upzoning > 200 is 387


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1016414,1326023,R-4-MHD (BASIS),mountain_view,TRA2,Mountain View - Whisman,35.0,3570.0,10200.0,1083.403039,10200.0
1137055,1279513,MFH_pba40,milpitas,TRA2,Milpitas - Transit Area Specific Plan,20.0,1800.0,9000.0,2781.884813,9000.0
1309887,1279523,MFH_pba40,milpitas,TRA2,Milpitas - Transit Area Specific Plan,20.0,1560.0,7800.0,3190.372977,7800.0
551025,2051469,RMED (BASIS),sunnyvale,TRA2,Sunnyvale - Lawrence Station Transit Village,24.0,840.0,3500.0,935.921778,3500.0
964657,981365,NCT (BASIS),san_francisco,TRA1,San Francisco - Eastern Neighborhoods,200.0,0.0,3000.0,1275.215439,3000.0


In [202]:
dua_upzoning_high_nongg = dua_upzoning.loc[(dua_upzoning.max_dua > 200) & (dua_upzoning.tra_id.isnull()) & (dua_upzoning.pda_id.isnull())]
print("Number of parcels with upzoning > 200 but not in a TRA or PDA is {}".format(len(dua_upzoning_high_nongg)))
dua_upzoning_high_nongg.head()

Number of parcels with upzoning > 200 but not in a TRA or PDA is 5


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
880673,1317393,R3-2.2 (BASIS),mountain_view,NaN,NaN,13.28,239.04,630.0,356.791058,630.0
941577,1317394,R3-2.2 (BASIS),mountain_view,NaN,NaN,13.28,212.48,560.0,356.791058,560.0
453813,1320529,P (BASIS),mountain_view,NaN,NaN,0.00,0.00,315.0,1700.800043,315.0
165232,1356371,RMED (BASIS),sunnyvale,NaN,NaN,24.00,192.00,280.0,935.921778,280.0
722088,1356372,RMED (BASIS),sunnyvale,NaN,NaN,24.00,144.00,210.0,935.921778,210.0


#### DUA from Imputation

In [213]:
dua_imputation = dua.loc[(dua.max_dua_imputation > dua.max_dua_zoning) & (dua.max_dua_imputation == dua.max_dua)]
print("Number of parcels with DUA > 40 from imputation {}".format(len(dua_upzoning.loc[dua_upzoning.max_dua > 200])))
dua_imputation.loc[dua_imputation.max_dua > 200].head()

Number of parcels with DUA > 40 from imputation 387


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
134280,900362,"NEIGHBORHOOD COMMERCIAL, MODERATE SCALE-130-V_...",san_francisco,TRA2,San Francisco - Downtown/Van Ness/Northeast Ne...,377.52,6417.84,6417.84,2154.205470,6417.84
1095012,98226,Mixed Use with Residential (MUR) (HE Data),emeryville,TRA3,Emeryville - Mixed-Use Core,502.00,2510.00,2510.00,3287.372932,2510.00
592285,1546598,DC (HE Data),san_jose,TRA2,San Jose - Greater Downtown,800.00,2400.00,2400.00,3602.408581,2400.00
1497090,1548250,Downtown_pba40,san_jose,TRA2,San Jose - Greater Downtown,800.00,2400.00,2400.00,3602.408581,2400.00
446376,99182,RHD-4-135-55_pba40,emeryville,TRA3,Emeryville - Mixed-Use Core,135.00,2025.00,2025.00,3287.372932,2025.00


In [204]:
dua_imputation_high_nongg = dua_imputation.loc[(dua_imputation.max_dua > 200) & (dua_imputation.tra_id.isnull()) & (dua_imputation.pda_id.isnull())]
print("Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is {}".format(len(dua_imputation_high_nongg)))
dua_imputation_high_nongg

Number of parcels with DUA > 40 from imputation and not in a TRA or PDA is 13


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
740058,1899033,RL_pba40,petaluma,NaN,NaN,8.0,632.0,632.0,37412.548179,632.0
1572269,1857256,UR (BASIS),rio_vista,NaN,NaN,2.5,620.0,620.0,2823.680759,620.0
307933,1802218,Planned Mixed Use (PMU) (HE Data),dixon,NaN,NaN,10.0,510.0,510.0,624.634900,510.0
1281746,357185,RES MED (BASIS),fremont,NaN,NaN,29.9,478.4,478.4,1860.826429,478.4
1480371,2043358,R-1-6 (BASIS),santa_rosa,NaN,NaN,6.0,414.0,414.0,8337.274413,414.0
1606280,438555,R-MD (BASIS),brentwood,NaN,NaN,11.0,374.0,374.0,1091.470952,374.0
1612118,1795666,Residential Low Density_pba40,vacaville,NaN,NaN,7.0,364.0,364.0,1347.429969,364.0
504096,1885665,R-2 (BASIS),santa_rosa,NaN,NaN,8.0,320.0,320.0,5354.811629,320.0
1076855,1365786,HDRD (BASIS),santa_clara,NaN,NaN,50.0,300.0,300.0,295.562077,300.0
1413812,2049431,Urban Residential_pba40,san_jose,NaN,NaN,95.0,285.0,285.0,1111.290428,285.0


#### DUA from Max TAZ Built DUA

In [205]:
dua_builtdua_taz_max = dua.loc[(dua.max_dua_builtdua_taz == dua.max_dua)]
print("Number of parcels with DUA > 40 set by imputation {}".format(len(dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40])))
dua_builtdua_taz_max.loc[dua_builtdua_taz_max.max_dua > 40].head()

Number of parcels with DUA > 40 set by imputation 5363


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1386909,1387559,Transit Residential_pba40,san_jose,TRA2,NaN,250.00,21750.00,21750.00,2118.716475,2118.716475
990881,982348,RH DTR (HE Data),san_francisco,TRA2,San Francisco - Transbay/Rincon Hill,1452.40,1452.40,1452.40,770.764431,770.764431
1670424,982303,CMUO (HE Data),san_francisco,TRA2,San Francisco - Eastern Neighborhoods,875.60,875.60,875.60,770.764431,770.764431
1251485,893460,RM-4 (HE Data),san_francisco,TRA2,San Francisco - Downtown/Van Ness/Northeast Ne...,680.00,680.00,680.00,624.140578,624.140578
981075,980933,PUBLIC-320-I_pba40,san_francisco,TRA1,San Francisco - Downtown/Van Ness/Northeast Ne...,929.28,929.28,929.28,486.154327,486.154327


In [206]:
dua_builtdua_taz_max_high_nongg = dua_builtdua_taz_max.loc[(dua_builtdua_taz_max.max_dua > 200) & 
                                  (dua_builtdua_taz_max.tra_id.isnull()) & (dua_builtdua_taz_max.pda_id.isnull())]
print("Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is {}".format(len(dua_builtdua_taz_max_high_nongg)))
dua_builtdua_taz_max_high_nongg.head()

Number of parcels with DUA > 40 from max built dua in TAZ and not in a TRA or PDA is 38


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
896389,906821,85' Height Allowed (HE Data),san_francisco,NaN,NaN,261.4,246.84,246.84,224.573268,224.573268
1379202,1040761,RM-4 (BASIS),san_francisco,NaN,NaN,217.0,217.00,217.00,210.950235,210.950235
909443,1040457,RM-4 (BASIS),san_francisco,NaN,NaN,217.0,217.00,217.00,210.950235,210.950235
280684,1040467,RM-4 (BASIS),san_francisco,NaN,NaN,217.0,217.00,217.00,210.950235,210.950235
1885050,1040454,RM-4 (BASIS),san_francisco,NaN,NaN,217.0,217.00,217.00,210.950235,210.950235


DUA from Zoning Input

In [207]:
dua_zoning = dua.loc[(dua.max_dua_zoning == dua.max_dua)]
print("Number of parcels with DUA > 40 from input zoning is {}".format(len(dua_zoning)))
dua_zoning.head()

Number of parcels with DUA > 40 from input zoning is 68113


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1597068,981784,RH DTR (HE Data),san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,1559.7,3826.062573,1559.7
502998,981681,TB DTR (HE Data),san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,1559.7,3826.062573,1559.7
169064,981785,RH DTR (HE Data),san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1559.7,1559.7,1559.7,3826.062573,1559.7
85050,981723,TB DTR (HE Data),san_francisco,TRA1,San Francisco - Transbay/Rincon Hill,1538.1,1538.1,1538.1,3826.062573,1538.1
1669360,982865,CMUO (HE Data),san_francisco,TRA2,San Francisco - Eastern Neighborhoods,1536.8,1536.8,1536.8,2052.537124,1536.8


In [208]:
dua_zoning_high_nongg = dua_zoning.loc[(dua_zoning.max_dua > 40) & 
                                       ((dua_zoning.tra_id.isnull()) & (dua_zoning.pda_id.isnull()))]
print("Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is {}".format(len(dua_zoning_high_nongg)))
dua_zoning_high_nongg

Number of parcels with DUA > 40 from input zoning and not in a TRA or PDA is 16072


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1220829,900063,85' Height Allowed (HE Data),san_francisco,NaN,NaN,243.9,243.9,243.9,254.429338,243.9
1867854,907077,85' Height Allowed (HE Data),san_francisco,NaN,NaN,243.9,243.9,243.9,966.966894,243.9
209288,900576,85' Height Allowed (HE Data),san_francisco,NaN,NaN,243.9,243.9,243.9,292.793462,243.9
1545360,900099,85' Height Allowed (HE Data),san_francisco,NaN,NaN,243.9,243.9,243.9,254.429338,243.9
1662747,900478,85' Height Allowed (HE Data),san_francisco,NaN,NaN,243.9,243.9,243.9,345.596402,243.9
...,...,...,...,...,...,...,...,...,...,...
1053836,961525,RH-1(D) (HE Data),san_francisco,NaN,NaN,40.1,40.1,40.1,103.926558,40.1
1396399,1040764,RH-1 (HE Data),san_francisco,NaN,NaN,40.1,40.1,40.1,210.950235,40.1
1513258,1017251,RH-1(D) (HE Data),san_francisco,NaN,NaN,40.1,40.1,40.1,86.359661,40.1
1433884,1004612,RH-1 (HE Data),san_francisco,NaN,NaN,40.1,40.1,40.1,152.136386,40.1


In [211]:
dua_zoning_high_nongg_nonbig3 = dua_zoning.loc[(dua_zoning.parcel_id.isin(dua_zoning_high_nongg.parcel_id)) & 
            (dua_zoning.juris != 'san_francisco') & (dua_zoning.juris != 'san_jose') & (dua_zoning.juris != 'oakland')]
print("Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is {}".format(len(dua_zoning_high_nongg_nonbig3)))
dua_zoning_high_nongg_nonbig3.head()

Number of parcels with DUA > 40 from zoning input and not in a TRA, PDA, or Big 3 is 3102


,parcel_id,zoning_name,juris,tra_id,pda_id,max_dua_zoning,max_dua_imputation,max_dua_imputation_upzoning,max_dua_builtdua_taz,max_dua
1045726,1080116,T5C (HE Data),south_san_francisco,NaN,NaN,140.0,140.0,140.0,951.196835,140.0
1005842,1080132,T5C (HE Data),south_san_francisco,NaN,NaN,140.0,140.0,140.0,951.196835,140.0
1935534,1080106,T5C (HE Data),south_san_francisco,NaN,NaN,140.0,140.0,140.0,951.196835,140.0
1269624,1080041,T5C (HE Data),south_san_francisco,NaN,NaN,140.0,140.0,140.0,951.196835,140.0
1269635,1080042,T5C (HE Data),south_san_francisco,NaN,NaN,140.0,140.0,140.0,951.196835,140.0


In [212]:
# export to view location of input zoning on a map
dua_zoning_high_nongg_nonbig3.to_csv("dua_zoning_high_nongg_nonbig3.csv")

### FAR